# Repsly trial data

In [ ]:
from repsly_data import RepslyData

repsly_data = RepslyData()
print('Reading data (this might take a minute or so)...', end='')
repsly_data.read_data('data/trial_users_analysis.csv', mode='FC')
print('done.')


Let's see what the data looks like:

In [ ]:
read_batch = repsly_data.read_batch(batch_size=20)

X, y = next(read_batch)
print('X{}: {}'.format(list(X.shape), X))
print('y:', y)

As you can see above, each input vector `X` has `1+15*16=241` values, most of which are zeros. The first one is the trial start date as offset from `2016-01-01` and the rest is different usage parameters for the following `16` days. Data provided by batch read is randomly shuffled. Output values are stored in `y` and they represent if the user purchased the Repsly service after the trial or not.

# Training

First, we create a network with two fully connected hidden layers of size 250 and 50% dropout:

In [ ]:
from repsly_nn import RepslyFC

repsly_nn = RepslyFC()

Then we train it for some number of epochs. (NB. there is s bug with restoring checkpoint)

In [ ]:
import numpy as np

arch = [250, 250]

keep_probs = np.linspace(0.4, 0.6, 11)
input_keep_probs = np.linspace(0.7, 1.0, 11)

learning_rate = 0.001
decay_steps=10
decay_rate=0.99

batch_size = 1024
epochs_loops = 1
epochs_at_once = 400
skip_steps=20

stats = {}

for keep_prob in keep_probs:
    for input_keep_prob in input_keep_probs:
        arch_dict = {'keep_prob': keep_prob, 'input_keep_prob': input_keep_prob}

        repsly_nn.create_net(arch, arch_dict, learning_rate, decay_steps, decay_rate)
        for i in range(epochs_loops):
            print('Training for {} epochs...'.format(epochs_at_once))
            stats[(keep_prob, input)] = repsly_nn.train(data=repsly_data, batch_size=batch_size, epochs=epochs_at_once, skip_steps=skip_steps)
            print(stats[(keep_prob, input)])

In [ ]:
print({key: stats[key]['f1_score'] for key in stats.keys()})